In [89]:
import re
import os
import json
import csv
import duckdb as db
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
# from pydantic import BaseModel
from typing import Any, Dict, List, Tuple, TypedDict, Annotated
from textwrap import dedent
# from crewai_tools import tool, FileWriterTool
from crewai import Agent, Crew, Process, Task
# from langchain_ollama.llms import OllamaLLM
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI

# CrewAI

In [2]:
# model = ChatGroq(
#     model="llama3-8b-8192",
#     temperature = 0,
# )

In [3]:
model = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [4]:
# model = ChatGoogleGenerativeAI(
#     model='gemini-1.5-pro-exp-0801',
#     temperature=0
# )

In [59]:
from os import listdir
from os.path import isfile, join

In [60]:
def query_list(file_path: str) -> List:
    with open(file_path) as f:
        s = f.read()
    
    return s

In [61]:
def load_queries(folder_path: str):
    file_list = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
    file_list.sort()
    queries = []  

    for sql_file in file_list:
        q = query_list(join(folder_path,sql_file))
        queries.append({
            'query_name': sql_file[:-4],
            'sql_code': q 
        })          

    return queries

In [62]:
path = 'sql_files/bn_beneficiario/'

In [63]:
queries = load_queries(path)

In [64]:
sql_analyst = Agent(
    role = "Senior Data Analyst",
    goal = "Analyze and complicate SQL queries and extract the relationship between table name and column name of all tables in queries.",
    backstory=dedent(
        """
        You're a highly specialized developed to dissect and understand complex SQL queries,
        you could quickly and accurately extract essential information from intricate SQL statements, 
        including that ones that has many sub queries.
        Your key traits are Analytical prowess, Attention to detail, Vast knowledge of SQL syntax 
        across multiple database systems.
        """
    ),
    llm=model,
    allow_delegation=False
)

In [65]:
get_table_names = Task(
    description=dedent(
        """ 
        Search this SQL query {sql_code} for all table names involved.
        It is very important not to ignore any tables. In complex queries, 
        there are some subqueries that must be observed carefully.
        Do it line by line, get all table names and their alias when they are present.
        """
    ),
    expected_output="List of distinct table names and alias present in the query.",
    agent=sql_analyst,
)

In [66]:
extract_tables_columns = Task(
    description=dedent(
        """
        Analyse this SQL querie {sql_code}.
        Use the list of table names and their aliases extracted in the previous step and find the columns for each of these tables.
        Then extract the names of related tables and columns. For this walkthrough, you will get all the table and column names used in this query.
        Get just the table name and column name by following these patterns:
        table_name;alias;columns_name
        table1;alias1;columnName_n1
        table1;alias1;columnName_n2
        table2;alias2;columnName_n1
        table2;alias2;columnName_n2
        tableN;aliasN;columnName_n1
        tableN;aliasN;columnName_n1
        """
    ),
    expected_output="CSV file",
    agent=sql_analyst,
    context=[get_table_names]
    #callback=lambda result: result_collector.add_result(result)
)

In [67]:
extract_rules = Task(
    description=dedent(
        """
        Analyse this SQL querie {sql_code}.
        Use the csv file generated in the previous step and find and extract following points:
        * SQL code where sql functions are used such as NVL, DECODE, CASE, SUM, etc, or concatenations lets call it of rules.
        * After rules we can find the alias for the column, the alias names are the final names given to the columns in the query. 
        Then step by step extract the column names, the alias, and the rule that gave rise to the alias.
        Get column_name, alias, and rule, following these patterns only for columns were you find rules:
        columns_name;alias;rule
        columnName_n1;alias_n1;"rule_n1"
        columnName_n2;alias_n2;"rule_n2"
        columnName_n3;alias_n3;"rule_n3"
        columnName_nn;alias_nn;"rule_nn"
        """
    ),
    expected_output="CSV file",
    agent=sql_analyst,
    context=[extract_tables_columns]
    #callback=lambda result: result_collector.add_result(result)
)

In [68]:
crew = Crew(
    agents = [sql_analyst],
    tasks = [get_table_names, extract_tables_columns, extract_rules],
    process = Process.sequential,
    verbose = 0,
    memory=False,
    output_log_file="crew.log",
)

2024-08-29 15:16:53,654 - 130117974093824 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [69]:
##self.export_to_csv()

def save_to_parquet(data, filename, header: list):
    # Split the string by newlines to get rows
    rows = data.split('\n')
    
    # Split each row by semicolons to get columns
    formatted_data = [row.split(';') for row in rows]
    
    # Output filename
    output_filename = f"{'output'}/{filename}.parquet"

    df = pd.DataFrame(formatted_data, columns=header)
    table = pa.Table.from_pandas(df)
    pq.write_table(table,output_filename)

In [78]:
for query in queries:
    print(query['query_name'])
    file_name = query['query_name']
    result = crew.kickoff(inputs=query)
    task_output = extract_tables_columns.output
    save_to_parquet(task_output.raw, file_name, ['table','alias','column_name'])
    task_output = extract_rules.output
    save_to_parquet(task_output.raw, file_name+'_rules', ['column_name','alias','rule'])

    #result_collector.add_result('export_tables_columns',output.raw,file_name)

In [83]:
rows = extract_rules.output.raw.split('\n')

In [84]:
formatted_data = [row.split(';') for row in rows]

In [87]:
df = pd.DataFrame(formatted_data, columns=['column_name','alias','rule'])

In [90]:
table = pa.Table.from_pandas(df)

In [91]:
pq.write_table(table,'output/teste.parquet')

# Processa CSVs

In [71]:
db.sql(
"""     
    with ben_ori as (
        select * from 'output/01_beneficiario.csv' 
        union all
        select * from 'output/02_sam_familia_teto_pf.csv' 
        union all
        select * from 'output/03_1_busca_microsiga.csv'
        union all
        select * from 'output/03_2_sem_setor.csv'
    ),
    ben_rul as (
        select 
            SUBSTR(column_name, INSTR(column_name, '.') + 1) AS column_name,
            alias,
            rule
        from 'output/01_beneficiario_rules.csv'
        union all
        select 
            SUBSTR(column_name, INSTR(column_name, '.') + 1) AS column_name,
            alias,
            rule
        from 'output/02_sam_familia_teto_pf_rules.csv' 
        union all
        select 
            SUBSTR(column_name, INSTR(column_name, '.') + 1) AS column_name,
            alias,
            rule
        from 'output/03_1_busca_microsiga_rules.csv'
        union all
        select 
            SUBSTR(column_name, INSTR(column_name, '.') + 1) AS column_name,
            alias,
            rule        
        from 'output/03_2_sem_setor_rules.csv'
    ),
    ben_dest as (
        select * from 'output/99_bn_beneficiario.csv'
    )
    select 
        distinct 
        ben_dest.*,
        ben_ori.*,
        ben_rul.*
    from ben_dest
    left join ben_ori 
    on(ben_ori.column_name = ben_dest.column_name)
    left join ben_rul
    on(ben_dest.colum_name_renamed = ben_rul.alias)
    order by ben_ori.column_name
""").to_csv('duckdb_output.txt')

BinderException: Binder Error: Set operations can only apply to expressions with the same number of result columns

In [75]:
db.sql(
"""     
        select 
            SUBSTR(column_name, INSTR(column_name, '.') + 1) AS column_name,
            alias,
            rule
        from 'output/01_beneficiario_rules.csv'
        union all
        select 
            SUBSTR(column_name, INSTR(column_name, '.') + 1) AS column_name,
            alias,
            rule
        from 'output/02_sam_familia_teto_pf_rules.csv' 
        union all
        select 
            SUBSTR(column_name, INSTR(column_name, '.') + 1) AS column_name,
            alias,
            rule
        from 'output/03_1_busca_microsiga_rules.csv'
        union all
        select 
            SUBSTR(column_name, INSTR(column_name, '.') + 1) AS column_name,
            alias,
            rule        
        from 'output/03_2_sem_setor_rules.csv'
""")

BinderException: Binder Error: Column "column_name" referenced that exists in the SELECT clause - but this column cannot be referenced before it is defined

In [93]:
db.sql("select * from 'output/teste.parquet'")

┌─────────────┬──────────────┬──────────────────────┐
│ column_name │    alias     │         rule         │
│   varchar   │   varchar    │       varchar        │
├─────────────┼──────────────┼──────────────────────┤
│ CTT_DESC01  │ SETOR_UNIMED │ "TRIM(B.CTT_DESC01)" │
│ RA_TELEFON  │ TELEFONE     │ "TRIM(A.RA_TELEFON)" │
└─────────────┴──────────────┴──────────────────────┘